In [1]:
import ROOT
import os
#os.environ["X509_USER_PROXY"] = "/eos/home-t/ttedesch/SWAN_projects/RDataFrame_porting/proxy"

Error in <TExMap::Add>: key 140182940188128 is not unique
Error in <TExMap::Add>: key 140182940704720 is not unique
TClassTable::Add:0: RuntimeWarning: class timespec already in TClassTable
TClassTable::Add:0: RuntimeWarning: class reverse_iterator<__gnu_cxx::__normal_iterator<TString*,vector<TString> > > already in TClassTable
TClassTable::Add:0: RuntimeWarning: class reverse_iterator<__gnu_cxx::__normal_iterator<int*,vector<int> > > already in TClassTable
TClassTable::Add:0: RuntimeWarning: class reverse_iterator<__gnu_cxx::__normal_iterator<string*,vector<string> > > already in TClassTable
TClassTable::Add:0: RuntimeWarning: class __gnu_cxx::__normal_iterator<char*,string> already in TClassTable
TClassTable::Add:0: RuntimeWarning: class __gnu_cxx::__normal_iterator<const char*,string> already in TClassTable
TClassTable::Add:0: RuntimeWarning: class __gnu_cxx::__normal_iterator<TString*,vector<TString> > already in TClassTable
TClassTable::Add:0: RuntimeWarning: class __gnu_cxx::__no

Welcome to JupyROOT 6.25/01


In [2]:
#ROOT.ROOT.EnableImplicitMT()

In [3]:
ROOT.gInterpreter.Declare('#include "preselection.h"')

ROOT.gInterpreter.Declare(
"""

float sample1_weight(){
    return 1.0f;
}

float sample2_weight(){
    return 2.0f;
}

float sample3_weight(){
    return 3.0f;
}

float samples_weights(unsigned int slot, const ROOT::RDF::RSampleInfo &id){
    if (id.Contains("sample1")){
        return sample1_weight();
    } else if (id.Contains("sample2")){
        return sample2_weight();
    } else if (id.Contains("sample3")){
        return sample3_weight();
    }
    return -999.0f;
}

std::string samples_names(unsigned int slot, const ROOT::RDF::RSampleInfo &id){
    return id.AsString();
}

"""
)

True

In [4]:
df = ROOT.RDataFrame("Events", [#"root://cms-xrd-global.cern.ch//store/user/apiccine/VBS/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_LL_SM_2017/210420_123836/0000/tree_hadd_1.root",
                                #"root://cms-xrd-global.cern.ch//store/user/apiccine/VBS/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/VBS_SSWW_LL_SM_2017/210420_123836/0000/tree_hadd_2.root",
                                "root://131.154.97.130.myip.cloud.infn.it:30443//store/mc/RunIIFall17NanoAODv7/VBS_SSWW_LL_polarization_TuneCP5_13TeV-madgraph-pythia8/NANOAODSIM/PU2017_12Apr2018_Nano02Apr2020_102X_mc2017_realistic_v8-v1/100000/5ADAED4E-FEEE-884F-8555-FF6FB31D6A60.root"]).Range(1000,2000)

In [5]:
#print(df.GetColumnNames())

In [6]:
#df.DefinePerSample("sample_weight", "samples_weights(rdfslot_, rdfsampleinfo_)").DefinePerSample("sample_name", "samples_names(rdfslot_, rdfsampleinfo_)").Display().Print()

df_year = df.DefinePerSample("Year","GetYear(rdfslot_, rdfsampleinfo_)")
#### MCWeight_writer ####


#### MET HLT Filter ####   #### working only for 2017 and 2018 (2016 requires a column that is not present in 2017 nd 2018, i.e. HLT_IsoTkMu24
df_METHLTFilter = df_year.Define("METHLTFilter", "MET_HLT_Filter(Year, Flag_goodVertices, Flag_HBHENoiseFilter, Flag_HBHENoiseIsoFilter, Flag_EcalDeadCellTriggerPrimitiveFilter, Flag_BadPFMuonFilter, Flag_globalSuperTightHalo2016Filter, HLT_Ele27_WPTight_Gsf,  HLT_Ele32_WPTight_Gsf, HLT_IsoMu24, HLT_IsoMu27, HLT_Mu50, HLT_Ele35_WPTight_Gsf, HLT_Ele32_WPTight_Gsf_L1DoubleEG, HLT_Photon200)")\
                         .Filter("METHLTFilter == true", "MET HLT Filter")

#### preselection #####
df_preselection = df_METHLTFilter.Filter("PV_ndof> 4 && abs(PV_z) < 20 && hypot(PV_x, PV_y)<2", "Good vertex")\
                                 .Define("HT_eventHT", "GetEventHT(Jet_pt, Jet_eta, Jet_phi, Jet_mass)")

#### lepSF #####
df_LepSF = df_year.Define("LepSF_output", "LepSF(Electron_pt, Electron_eta, Electron_pdgId, Muon_pt, Muon_eta, Muon_pdgId, Year)")\
                          .Define("Muon_effSF", "LepSF_output[0]")\
                          .Define("Electron_effSF", "LepSF_output[1]")\
                          .Define("Muon_effSF_errUp", "LepSF_output[2]")\
                          .Define("Electron_effSF_errUp", "LepSF_output[3]")\
                          .Define("Muon_effSF_errDown", "LepSF_output[4]")\
                          .Define("Electron_effSF_errDown", "LepSF_output[5]")

#### puWeight #####


#### btagSF #####


#### prefCorr ####


#### metCorrector ####


#### fatJetCorrector ####


#### muonScaleRes ####


#### ht ####


#### mht ####
df_mht = df.Define("MHT_pt_phi", "MHT_pt_phi(Electron_pt, Electron_eta, Electron_phi, Electron_mass, Electron_miniPFRelIso_all, Muon_pt, Muon_eta, Muon_phi, Muon_mass, Muon_miniPFRelIso_all, Jet_pt, Jet_eta, Jet_phi, Jet_mass, Jet_muonIdx1, Jet_muonIdx2, Jet_electronIdx1, Jet_electronIdx2, nJet)")\
           .Define("MHT_pt", "getFirst(MHT_pt_phi)").Define("MHT_phi", "getSecond(MHT_pt_phi)")

Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available


In [7]:
#df_mht.Display("MHT_pt_phi").Print()

In [8]:
df.Report().Print()

In [9]:
df_LepSF.Display("Electron_effSF").Print()

Loading histogram NUM_TightID_DEN_genTracks_pt_abseta from file leptonSF/Muon_RunBCDEF_SF_ID_2017.root... 
Loading histogram NUM_TightRelIso_DEN_TightIDandIPCut_pt_abseta from file leptonSF/Muon_RunBCDEF_SF_ISO_2017.root... 
Loading histogram EGamma_SF2D from file leptonSF/EGM2D_2017_passingRECO_highEt.root... 
Loading histogram EGamma_SF2D from file leptonSF/Electron_MVA90_2017.root... 
Loading histogram NUM_TightID_DEN_genTracks_pt_abseta from file leptonSF/Muon_RunBCDEF_SF_ID_2017.root... 
Loading histogram NUM_TightRelIso_DEN_TightIDandIPCut_pt_abseta from file leptonSF/Muon_RunBCDEF_SF_ISO_2017.root... 
Loading histogram EGamma_SF2D from file leptonSF/EGM2D_2017_passingRECO_highEt.root... 
Loading histogram EGamma_SF2D from file leptonSF/Electron_MVA90_2017.root... 
Loading histogram NUM_TightID_DEN_genTracks_pt_abseta from file leptonSF/Muon_RunBCDEF_SF_ID_2017.root... 
Loading histogram NUM_TightRelIso_DEN_TightIDandIPCut_pt_abseta from file leptonSF/Muon_RunBCDEF_SF_ISO_2017.ro

In [10]:
df_LepSF.Display("Electron_effSF_errUp").Print()

Loading histogram NUM_TightID_DEN_genTracks_pt_abseta from file leptonSF/Muon_RunBCDEF_SF_ID_2017.root... 
Loading histogram NUM_TightRelIso_DEN_TightIDandIPCut_pt_abseta from file leptonSF/Muon_RunBCDEF_SF_ISO_2017.root... 
Loading histogram EGamma_SF2D from file leptonSF/EGM2D_2017_passingRECO_highEt.root... 
Loading histogram EGamma_SF2D from file leptonSF/Electron_MVA90_2017.root... 
Loading histogram NUM_TightID_DEN_genTracks_pt_abseta from file leptonSF/Muon_RunBCDEF_SF_ID_2017.root... 
Loading histogram NUM_TightRelIso_DEN_TightIDandIPCut_pt_abseta from file leptonSF/Muon_RunBCDEF_SF_ISO_2017.root... 
Loading histogram EGamma_SF2D from file leptonSF/EGM2D_2017_passingRECO_highEt.root... 
Loading histogram EGamma_SF2D from file leptonSF/Electron_MVA90_2017.root... 
Loading histogram NUM_TightID_DEN_genTracks_pt_abseta from file leptonSF/Muon_RunBCDEF_SF_ID_2017.root... 
Loading histogram NUM_TightRelIso_DEN_TightIDandIPCut_pt_abseta from file leptonSF/Muon_RunBCDEF_SF_ISO_2017.ro

In [ ]:
df_LepSF.Display("Muon_effSF").Print()

In [ ]:
df_LepSF.Display("Muon_effSF_errUp").Print()